# Project Part 3

[![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://www.kaggle.com/code/dereckhelms/cs39aa-project-part-2)

Colab link is a work in progress

## 1. Creation of Feed-Forward Model
Although I do not believe a feed-forward model is the most appropriate for this set of structured data, I want to construct one to see how it behaves in relation to another deep learning model. 
Below are the imports I will use for this section.
It is necessary to convert the binary smoker feature to one hot encoding for the model to analyze it without errors. StandardScaler is used to scale the data to a normal distribution and if all the data is not of float type I will receive many errors.

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load the dataset
df = pd.read_csv('insurance.csv')

# Extract features and target variable
features = df[['age', 'bmi', 'children', 'smoker', 'region']]
target = df['charges']

# Explicitly create a copy of the DataFrame to avoid SettingWithCopyWarning
features = features.copy()

# Convert categorical features to one-hot encoding
label_encoder = LabelEncoder()
features['smoker'] = label_encoder.fit_transform(features['smoker'])
features = pd.get_dummies(features, columns=['region'])

# Standardize numerical features
scaler = StandardScaler()
features[['age', 'bmi', 'children']] = scaler.fit_transform(features[['age', 'bmi', 'children']])

# Ensure all data is of numeric type
features = features.astype('float32')
target = target.astype('float32')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Define the model architecture
class InsuranceModel(nn.Module):
    def __init__(self, input_size):
        super(InsuranceModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # Adjust the number of hidden units as needed
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)  # Output is a single number for insurance charge prediction

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Create a model instance
input_size = X_train_tensor.shape[1]
model = InsuranceModel(input_size)

# Set up the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust the learning rate as needed

# Training loop
num_epochs = 1000  # Adjust as needed
for epoch in range(num_epochs):
    # Forward pass
    predictions = model(X_train_tensor)
    
    # Compute the loss
    loss = criterion(predictions, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print training information
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_predictions = model(X_test_tensor)
    test_loss = criterion(test_predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

# Print some predicted versus actual values
with torch.no_grad():
    model.eval()
    sample_indices = np.random.choice(len(X_test_tensor), 5, replace=False)
    for i in sample_indices:
        prediction = model(X_test_tensor[i])
        print(f'Sample {i + 1}: Predicted={prediction.item():.4f}, Actual={y_test_tensor[i].item():.4f}')



ModuleNotFoundError: No module named 'torch'

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have already trained the model and obtained predictions
with torch.no_grad():
    model.eval()
    predictions = model(X_test_tensor)

# Convert torch tensors to numpy arrays for visualization
y_pred = predictions.numpy()
y_actual = y_test_tensor.numpy()

# Create separate scatter plots for each feature

# Plot for BMI
plt.scatter(X_test_tensor[:, 2].numpy(), y_actual, color='red', label='Actual Charges')
plt.scatter(X_test_tensor[:, 2].numpy(), y_pred, color='blue', label='Predicted Charges')
plt.xlabel("BMI")
plt.ylabel("Charges")
plt.title("BMI: Actual vs Predicted Charges")
plt.legend()
plt.show()

# Plot for Smoker
plt.scatter(X_test_tensor[:, 1].numpy(), y_actual, color='red', label='Actual Charges')
plt.scatter(X_test_tensor[:, 1].numpy(), y_pred, color='blue', label='Predicted Charges')
plt.xlabel("Smoker")
plt.ylabel("Charges")
plt.title("Smoker: Actual vs Predicted Charges")
plt.legend()
plt.show()

# Plot for Regions
for region_index in range(3, 7):  # Assuming regions are columns 3 to 6
    plt.scatter(X_test_tensor[:, region_index].numpy(), y_actual, color='red', label='Actual Charges')
    plt.scatter(X_test_tensor[:, region_index].numpy(), y_pred, color='blue', label='Predicted Charges')
    plt.xlabel(f"Region {region_index - 2}")
    plt.ylabel("Charges")
    plt.title(f"Region {region_index - 2}: Actual vs Predicted Charges")
    plt.legend()
    plt.show()

# Plot for Children
plt.scatter(X_test_tensor[:, 4].numpy(), y_actual, color='red', label='Actual Charges')
plt.scatter(X_test_tensor[:, 4].numpy(), y_pred, color='blue', label='Predicted Charges')
plt.xlabel("Children")
plt.ylabel("Charges")
plt.title("Children: Actual vs Predicted Charges")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error

# ... (previous code)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_actual, y_pred)
print(f'Mean Absolute Error: {mae:.2f}')

# Continue with the rest of the code for scatter plots


This is the first attempt at generating a feed forward model. As one can observe my model predicts insurance charges that are approximately thousands of dollars off. Perhaps adding another neural network layer will improve the model's accuracy. 

## 2. Fine tuning of Feed-Forward Model
Below I added more layers to the model and added training and validation sets. I've added dropout layers and learning rate. I plan to change these hyperparameters to see how they affect the model's MAE.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have a dataset named 'data' with features and labels
# Features include: age, smoker, bmi, children, region

# Example data (replace this with your actual data)
data = {
    'age': np.random.rand(100, 1),
    'smoker': np.random.rand(100, 1),
    'bmi': np.random.rand(100, 1),
    'children': np.random.rand(100, 1),
    'region': np.random.rand(100, 1),
    'charges': np.random.rand(100, 1),  # Assuming 'charges' is the target variable
}

# Combine features into a single input tensor
features = np.hstack([data['age'], data['smoker'], data['bmi'], data['children'], data['region']])
target = data['charges']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the model architecture
class FeedForwardModel(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size, dropout_rate, learning_rate):
        super(FeedForwardModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Instantiate the model
input_size = X_train.shape[1]
output_size = 1  # Assuming regression task
hidden_size1 = 64
hidden_size2 = 32
dropout_rate = 0.2
learning_rate = 0.001  # Set your desired learning rate
model = FeedForwardModel(input_size, hidden_size1, hidden_size2, output_size, dropout_rate, learning_rate)

# Set up the loss function and optimizer
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
num_epochs = 1000  # Adjust as needed
for epoch in range(num_epochs):
    # Forward pass
    predictions = model(X_train_tensor)
    
    # Compute the loss
    loss = criterion(predictions, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print training information
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model on the test set
with torch.no_grad():
    model.eval()
    test_predictions = model(X_test_tensor)
    mae_test = mean_absolute_error(y_test_tensor, test_predictions)
    print(f"Mean Absolute Error on Test Set: {mae_test}")

# Visualize results
plt.scatter(y_test, test_predictions, color='blue')
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red', linewidth=2)
plt.xlabel('True Charges')
plt.ylabel('Predicted Charges')
plt.title('True vs Predicted Charges')
plt.show()
